# Interactive LLM Notebook

this notebook provides the front end for interacting with the context-given LLM. 

In [1]:
import os
# import param
import panel as pn
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
pn.extension()


In [2]:
from utils.utils import get_openai_api_key

In [3]:
os.environ['OPENAI_API_KEY'] = get_openai_api_key('api_key.txt')
##need to load this first... before we load in the config... need to think about HOW this should work

In [4]:
## Eventually make this something to be loaded 
config = {
    'basedir': '../vectorstore/',
    'retrain_str': False,
    'persist_dir': '../vectorstore/chroma/',
    'datadir': '../../journals/',
    'text_splitter': RecursiveCharacterTextSplitter(
                            chunk_size = 1500,
                            chunk_overlap = 150,
                            separators=['<|endoftext|>',"\n\n", "\n", "\n(.+?)\n","(?<=\. )", " ", ""]
                                            ),
    'embeddings':OpenAIEmbeddings(),
    'allowed_special_kwargs': '<|endoftext|>',
    'llm': ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0), }


In [5]:
from utils.utils import retrieve_context_vectordb, cbfs

In [6]:
config['retriever'] = retrieve_context_vectordb(config)

loading vectordb


In [10]:
cb = cbfs(config)


In [11]:
file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp) 

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# DocuBot: Chat with your Documents')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),)
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height=300, loading_indicator=True)
            [3] Divider()
        [1] Column
            [0] ParamMethod(method, _pane=Column, defer_load=False)
            [1] Divider()
            [2] ParamMethod(method, _pane=Str, defer_load=False)
        [2] Column
            [0] ParamMethod(method, _pane=WidgetBox, defer_load=False)
            [1] Divider()

## to do next:
5. config branch to write config and build_config.py script
6. build_config.py should also generate a .txt for the api_key which can be read it seperately and before utils... (?)
4. write in readme.md about project
5. finish